In [ ]:
# libraries needed to get data from https://api.themoviedb.org/...
import requests
import json

import re
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

Resources: https://youtu.be/Sg5VTTBIhqo
           https://github.com/codingforentrepreneurs/30-Days-of-Python/blob/master/tutorial-reference/Day%2013/connect.py

In [ ]:
df = pd.read_csv("movie_metadata.csv")

# create a database with movie titles and years to use with API to look up information
name_list = df[['movie_title', 'title_year']]
name_list = name_list[name_list['title_year'] >= 1999]
name_list = name_list.drop_duplicates()
name_list = name_list.astype({'title_year': int, 'movie_title': str})
name_list = name_list.astype({'title_year': str, 'movie_title': str})
name_list.reset_index(drop=True, inplace=True)

In [ ]:
def movie_matching(title: str, _id: str, j: int) -> bool:
    ''' checks if the movie found matchs the title of the movie in the database'''
    if (not re.match(title, name_list.iloc[j, 0])):
        print(f'no match: {title}, {name_list.iloc[j, 0]}, {j}')
        if j == (len(name_list) - 1): #stop checking at 2000 to avoid stack overrun
            j = 0
            return False;
        else:
            j += 1
            movie_matching(title, _id, j)
    else: 
        movie_ids.add(_id)
        print(f'match: {title}, {name_list.iloc[j, 0]}, {j}')
        # if found, drop title from list to reduce search time and so remaining titles can be searched for again without the year
        name_list.drop(j, inplace = True) 
        name_list.reset_index(drop=True, inplace=True)
        return True;

In [ ]:
def movie_lookup(name: str, year: int) -> None:
    ''' given a name and year, looks up movie index using themoviedb.org api'''
    endpoint = ""
    api_key = "e324e12054790b52bb68c8d98118dd96"
    api_version = 3
    api_base_url = f"https://api.themoviedb.org/{api_version}"
    endpoint_path = f"/search/movie"
    if year != 0:
        endpoint = f"{api_base_url}{endpoint_path}?api_key={api_key}&query={name}&year={year}"
    else:
        endpoint = endpoint = f"{api_base_url}{endpoint_path}?api_key={api_key}&query={name}"
    r = requests.get(endpoint) 
    if r.status_code in range(200, 299):
        data = r.json()
        results = data['results']
        found = False
        print(len(results))
        if len(results) > 0:
            for result in results: 
                title = result['title']
                _id = result['id']
                if(movie_matching(title, _id, 0)):
                    found = True
                    break;    

In [ ]:
movie_ids = set() #holds ids found on movie database that match with title from origial database


#loop through movies in name_List and look up their ids
for i in range(len(name_list)) :
    name = name_list.iloc[i, 0]
    year = name_list.iloc[i, 1]
    movie_lookup(name, year)


#search through unfound movies without submitting a year
for index, name, year in name_list.itertuples():
    movie_lookup(name, 0)

https://www.askpython.com/python/examples/detection-removal-outliers-in-python

In [ ]:
# if the program stops before running through all of items, 
#this can be used to combine the new data with previously saved data

old_movies = pd.read_csv("movie_ids.csv")
movie_ids_pd = pd.DataFrame(movie_ids)
together = [old_movies, movie_ids_pd]
combined = pd.concat(together)

#check combined in longer before saving in next step
print(len(old_movies))
print(len(combined))

In [ ]:
# Save any returned data so it won't need to be looked up again
movie_ids_csv = pd.DataFrame(combined)
movie_ids_csv.to_csv("movie_ids.csv", index=False)
# Save names of movies where no matches were found so they can be researched further
no_matches = pd.DataFrame(name_list)
no_matches.to_csv("movies_no_matches.csv", index=False)

In [ ]:
#use the ids found above to lookup budget and revenue data
movie_data = []
movie_ids = pd.read_csv("movie_ids.csv")
for index, movie_id in movie_ids.itertuples():
    api_version = 3
    api_key = "e324e12054790b52bb68c8d98118dd96"
    api_base_url = f"https://api.themoviedb.org/{api_version}"
    endpoint_path = f"/movie/{movie_id}"
    endpoint = f"{api_base_url}{endpoint_path}?api_key={api_key}"
    r = requests.get(endpoint)
    if r.status_code in range(200, 299):
        #drop ids so if the connection times out, the remaining ids can be saved and used
        movie_ids.drop(index, inplace = True) 
        data = r.json()
        movie_data.append([data['original_title'], data['budget'], data['revenue']])   
        print([data['original_title'], data['budget'], data['revenue']])

In [ ]:
# use if the connection times out
movie_data = pd.DataFrame(movie_data)
movie_ids.to_csv("movie_ids2")

In [ ]:
#use only if the connection times out to save obtained data
movies1 = pd.read_csv("movie_profits.csv")
movie_data.columns = ["title", "budget", "revenue"]
movies = [movie1, movie_data]
movie_budget = movies1.append(movie_data)
movie_budget.to_csv("movie_profits.csv", index = False)

In [ ]:
#calculate profits and add to dataframe, save as csv
movie_budget["profit"] = movie_budget["revenue"] - movie_budget["budget"]
movie_budget.to_csv("movie_profits.csv", index = False)